<a href="https://colab.research.google.com/github/ellamcho/NB240/blob/main/MNIST/20250204_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
# Load packages
import torch
from torch import nn
from torch import optim
from torchvision import datasets, transforms
from torch.utils.data import random_split, DataLoader

In [5]:
# Define the model
model = nn.Sequential(
    nn.Linear(28 * 28, 64),
    nn.ReLU(),
    nn.Linear(64, 64),
    nn.ReLU(),
    nn.Linear(64, 10)
)

In [8]:
# Define the optimizer
params = model.parameters
optimizer = optim.SGD(params(),lr = 1e-2)

In [9]:
# Define loss
loss = nn.CrossEntropyLoss()

In [20]:
# Training sets

train_data = datasets.MNIST('data', train=True, download=True, transform=transforms.ToTensor())
train, val = random_split(train_data, [55000, 5000]) # splits data randomly for training and then validation ("val")
train_loader = DataLoader(train, batch_size =32)
val_loader = DataLoader(val, batch_size = 32)

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 9.91M/9.91M [00:00<00:00, 43.7MB/s]


Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 28.9k/28.9k [00:00<00:00, 1.36MB/s]


Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 1.65M/1.65M [00:00<00:00, 10.7MB/s]


Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 4.54k/4.54k [00:00<00:00, 3.42MB/s]

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw



In [11]:
# Training loops
num_epochs = 5
for epoch in range(num_epochs):
  for batch in train_loader:
    x, y = batch

    # x: b x 1 x 28 x 28
    b = x.size(0)
    x = x.view(b, -1)

    # 1: Forward (l : logits)
    l = model(x)

    # 2: Compute the objective function
    J = loss(1, y)

    # 3: Clean the gradients
    model.zero_grad()
    #params.grad._zero()

    # 4: Accumulate the partial derivatives of J
    J.backward()
    # params.grad._sum(dJ/dparams)

    # 5: Step in the opposite direction of the gradient
    optimizer.step()

    # with torch_nograd: params = eta - params.grad()


NameError: name 'train_loader' is not defined